In [ ]:
import matplotlib.pyplot as plt;
from math import sqrt;
from matplotlib import pyplot;
import pandas as pd;
from numpy import concatenate;
import numpy as np;
from sklearn.preprocessing import MinMaxScaler;
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,roc_curve,auc,matthews_corrcoef, f1_score;
from keras.models import Sequential;
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation;
from keras.optimizers import Adam
from binn import BINN, Network, SuperLogger,BINNExplainer,ImportanceNetwork;
import pandas as pd;
from sklearn import preprocessing;
from typing import Union;
import torch;
from pytorch_lightning import Trainer
import torch.nn.functional as F
import joblib
from sklearn.linear_model import LinearRegression
from sksurv.metrics import concordance_index_censored
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from lifelines.utils import concordance_index
import xgboost as xgb
import statistics
import shap
def fit_data_matrix_to_network_input(data_matrix: pd.DataFrame, features, feature_column="Protein") -> pd.DataFrame:
    nr_features_in_matrix = len(data_matrix.index)
    if len(features) > nr_features_in_matrix:
        features_df = pd.DataFrame(features, columns=[feature_column])
        data_matrix = data_matrix.merge(
            features_df, how='right', on=feature_column)
    if len(features) > 0:
        data_matrix.set_index(feature_column, inplace=True)
        data_matrix = data_matrix.loc[features]
    return data_matrix


def generate_data(data_matrix: pd.DataFrame, design_matrix: pd.DataFrame):
    GroupOneCols = design_matrix[design_matrix['group']
                                 == 0]['sample'].values
    GroupTwoCols = design_matrix[design_matrix['group']
                                 == 1]['sample'].values

    df1 = data_matrix[GroupOneCols].T
    df2 = data_matrix[GroupTwoCols].T
    y = np.array([0 for _ in GroupOneCols] + [1 for _ in GroupTwoCols])
    X = pd.concat([df1, df2]).fillna(0).to_numpy()
    X = preprocessing.StandardScaler().fit_transform(X)
    return X, y

input_data = pd.read_csv("./data/SKCM144/pre-processing/brca_maf13.txt",sep="\t");
translation = pd.read_csv("./data/SKCM144/pre-processing/reactome_data13.txt",sep="\t");
pathways = pd.read_csv("./data/pathways.tsv", sep="\t");
network = Network(input_data=input_data,pathways=pathways,mapping=translation,input_data_column = "Protein",source_column = "child",target_column = "parent")
torch.manual_seed(0)
binn = BINN(network=network,n_layers=4,dropout=0.5,validate=True,residual=False,learning_rate=0.001)
design_matrix = pd.read_csv('./data/SKCM144/pre-processing/sample_data13.txt',sep="\t")
protein_matrix = fit_data_matrix_to_network_input(input_data, features=binn.features)
X, Y = generate_data(protein_matrix, design_matrix=design_matrix)

GroupOneCols = design_matrix [design_matrix ['group']== 0]['sample'].values
GroupTwoCols = design_matrix [design_matrix ['group']== 1]['sample'].values
a_110=GroupOneCols.tolist()+GroupTwoCols.tolist()
#Random forest model
model_RF = RandomForestClassifier(n_estimators=100, random_state=999)
model_RF.fit(X, Y)
y_pre_rf = model_RF.predict(X)
inv_y=Y
fpr,tpr,thersholds=roc_curve(Y, y_pre_rf )
roc_auc= auc(fpr,tpr)
print('Train AUC of random forest: %.3f' % roc_auc)
y_pre_rf=y_pre_rf.ravel()
c_index = concordance_index(inv_y, y_pre_rf)
print('Train C-index of random forest: %.3f' % c_index)
mcc = matthews_corrcoef(inv_y,y_pre_rf )
print('Train MCC of random forest: %.3f' % mcc)
F1_score = f1_score(inv_y,y_pre_rf )
print('Train F1_score of random forest: %.3f' % F1_score)


test_da2=pd.read_csv('./data/SKCM30/pre-processing/test_data_13.txt',sep="\t")
test_sam2=pd.read_csv('./data/SKCM30/pre-processing/sample_test_13.txt',sep="\t")
protein_matrix2 = fit_data_matrix_to_network_input(test_da2, features=binn.features)
test_x,test_y = generate_data(protein_matrix2, design_matrix=test_sam2)
y_pre_rf1 = model_RF.predict(test_x)
fpr,tpr,thersholds=roc_curve(test_y, y_pre_rf1)
roc_auc= auc(fpr,tpr)
print('Test AUC of random forest: %.3f' % roc_auc)
y_pre_rf1=y_pre_rf1.ravel()
c_index = concordance_index(test_y, y_pre_rf1)
print('Test C-index of random forest: %.3f' % c_index)
mcc = matthews_corrcoef(test_y,y_pre_rf1)
print('Test MCC of random forest: %.3f' % mcc)
F1_score = f1_score(test_y,y_pre_rf1)
print('Test F1_score of random forest: %.3f' % F1_score)



#SVM
model_SVM = SVC(kernel="rbf",random_state=999)
model_SVM.fit(X, Y)
y_pre_svm = model_SVM.predict(X)
inv_y=Y
fpr,tpr,thersholds=roc_curve(Y, y_pre_svm)
roc_auc= auc(fpr,tpr)
print('Train AUC of support vector machine: %.3f' % roc_auc)
y_pre_svm=y_pre_svm.ravel()
c_index = concordance_index(inv_y, y_pre_svm)
print('Train C-index of support vector machine: %.3f' % c_index)
mcc = matthews_corrcoef(inv_y,y_pre_svm)
print('Train MCC of support vector machine: %.3f' % mcc)
F1_score = f1_score(inv_y,y_pre_svm )
print('Train F1_score of support vector machine: %.3f' % F1_score)

y_pre_svm1 = model_SVM.predict(test_x)
fpr,tpr,thersholds=roc_curve(test_y, y_pre_svm1)
roc_auc= auc(fpr,tpr)
print('Test AUC of support vector machine: %.3f' % roc_auc)
y_pre_svm1=y_pre_svm1.ravel()
c_index = concordance_index(test_y, y_pre_svm1)
print('Test C-index of support vector machine: %.3f' % c_index)
mcc = matthews_corrcoef(test_y,y_pre_svm1)
print('Test MCC of support vector machine: %.3f' % mcc)
F1_score = f1_score(test_y, y_pre_svm1)
print('Test F1_score of support vector machine: %.3f' % F1_score)

#xgboost
dtrain = xgb.DMatrix(X, label=Y)
dtest = xgb.DMatrix(X, label=Y)
param = {'max_depth': 6,'eta': 0.3,  'objective': 'binary:logistic',  'eval_metric': 'logloss' }
num_round = 100
bst = xgb.train(param, dtrain, num_round)
y_pre_xgb = bst.predict(dtest)
fpr,tpr,thersholds=roc_curve(Y, y_pre_xgb)
roc_auc= auc(fpr,tpr)
print('Train AUC of XGBoost: %.3f' % roc_auc)
y_pre_xgb=y_pre_xgb.ravel()
c_index = concordance_index(inv_y, y_pre_xgb)
print('Train AUC of XGBoost: %.3f' % c_index)
median_value = statistics.median(y_pre_xgb)
binary_classification = np.where(y_pre_xgb < median_value, 0, 1)
mcc = matthews_corrcoef(inv_y,binary_classification )
print('Train MCC of XGBoost: %.3f' % mcc)
F1_score = f1_score(inv_y, binary_classification )
print('Train F1_score of XGBoost: %.3f' % F1_score)


dtest=xgb.DMatrix(test_x, label=test_y)
y_pre_xgb1 = bst.predict(dtest)
fpr,tpr,thersholds=roc_curve(test_y, y_pre_xgb1)
roc_auc= auc(fpr,tpr)
print('Test AUC of XGBoost: %.3f' % roc_auc)
y_pre_xgb1=y_pre_xgb1.ravel()
c_index = concordance_index(test_y, y_pre_xgb1)
print('Test C-index of XGBoost: %.3f' % c_index)
median_value = statistics.median(y_pre_xgb1)
binary_classification = np.where(y_pre_xgb1 < median_value, 0, 1)
mcc = matthews_corrcoef(test_y,binary_classification )
print('Test MCC of XGBoost: %.3f' % mcc)
F1_score = f1_score(test_y, binary_classification )
print('Test F1_score of XGBoost: %.3f' % F1_score)


#FNN
train_x=X
train_y=Y
model = Sequential()  
input = X.shape[1]
model.add(Dense(128, input_shape=(input,))) 
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(1))
# 使用高效的ADAM优化算法以及优化的最小均方误差损失函数
model.compile(loss='mean_squared_error', optimizer=Adam())
# 使用EarlyStopping来提前结束迭代次数，防止过拟合
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=50, verbose=2)
# 训练
torch.manual_seed(2)
history = model.fit(train_x, train_y, epochs=3000, batch_size=36, verbose=2,
                    shuffle=False, callbacks=[early_stopping])

test_x=X
test_y=Y
# 预测
yhat = model.predict(test_x)
inv_yhat=yhat 
inv_y=test_y
fpr,tpr,thersholds=roc_curve(Y, inv_yhat )
roc_auc= auc(fpr,tpr)
print('Train AUC of fully connected neural network: %.3f' % roc_auc)
inv_yhat=inv_yhat.ravel()
c_index = concordance_index(Y, inv_yhat)
print('Train C-index of fully connected neural network: %.3f' % c_index)
median_value = statistics.median(inv_yhat)
binary_classification = np.where(inv_yhat < median_value, 0, 1)
mcc = matthews_corrcoef(Y,binary_classification )
print('Train MCC of fully connected neural network: %.3f' % mcc)
F1_score = f1_score(Y, binary_classification )
print('Train F1_score of fully connected neural network: %.3f' % F1_score)

test_da2=pd.read_csv('./data/SKCM30/pre-processing/test_data_12.txt',sep="\t")
test_sam2=pd.read_csv('./data/SKCM30/pre-processing/sample_test_12.txt',sep="\t")
protein_matrix2 = fit_data_matrix_to_network_input(test_da2, features=binn.features)
test_x,test_y = generate_data(protein_matrix2, design_matrix=test_sam2)
yhat = model.predict(test_x)
fpr,tpr,thersholds=roc_curve(test_y, yhat)
roc_auc= auc(fpr,tpr)
print('Test AUC of fully connected neural network: %.3f' % roc_auc)
yhat=yhat.ravel()
c_index = concordance_index(test_y, yhat)
print('Test C-index of fully connected neural network: %.3f' % c_index)
median_value = statistics.median(yhat)
binary_classification = np.where(yhat < median_value, 0, 1)
mcc = matthews_corrcoef(test_y,binary_classification )
print('Test MCC of fully connected neural network: %.3f' % mcc)
F1_score = f1_score(test_y, binary_classification )
print('Test F1_score of fully connected neural network: %.3f' % F1_score)
